In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import chromedriver_autoinstaller
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains

import pandas as pd

## 현대 자동차 FAQ 크롤링

In [3]:
def crawl_one_quetion(driver, wait, action, idx) -> dict :
    result = {}

    # 질문 버튼 찾기
    button = driver.find_element(By.XPATH, f'//*[@id="app"]/div[3]/section/div[2]/div/div[2]/section/div/div[3]/div[1]/div[{idx}]/button')
    # 버튼이 클릭이 가능하도록 해당 위치로 스크롤
    action.move_to_element(button).perform()
    # 버튼이 클릭 가능해질 때 까지 대기
    button = wait.until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="app"]/div[3]/section/div[2]/div/div[2]/section/div/div[3]/div[1]/div[{idx}]/button')))
    # 버튼 클릭
    button.click()
    # 2초 대기
    time.sleep(2)
    
    # 질문 추출
    result['quetion'] = button.text

    # 답변 요소 찾기
    elem = driver.find_element(By.XPATH, f'//*[@id="app"]/div[3]/section/div[2]/div/div[2]/section/div/div[3]/div[1]/div[{idx}]/div')
    # 답변 추출
    result['answer'] = elem.text

    # 질문/답변 딕셔너리를 결과 리스트에 추가
    return result

def crawl_one_page(driver, wait, action):
    result = []

    for idx in range(1, 11):
            
        try:
            dict_elem = crawl_one_quetion(driver, wait, action, idx)
            
            # 질문/답변 딕셔너리를 결과 리스트에 추가
            result.append(dict_elem)

        except Exception as e:
            print(f'[ faild FAQ id > {idx} ]')
            print("*"*20)
    
    return result

def move_to_next_page(driver, wait, action, page):
    # 페이지 버튼을 찾기
    button = driver.find_element(By.XPATH, f'//*[@id="app"]/div[3]/section/div[2]/div/div[2]/section/div/div[3]/div[2]/div/ul/li[{page}]/button')
    # 페이지 버튼이 보이도록 위치 스크롤
    action.move_to_element(button).perform()
    # 페이비 버튼이 클릭 가능할 때 까지 대기
    button = wait.until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="app"]/div[3]/section/div[2]/div/div[2]/section/div/div[3]/div[2]/div/ul/li[{page}]/button')))

    # 페이지 버튼 클릭하여 다음 페이지로 이동
    button.click()
    # 2초 대기
    time.sleep(2)

In [4]:
chromedriver_autoinstaller.install()
driver = webdriver.Chrome()
action = ActionChains(driver)

# 크롤링할 웹 페이지 URL
url = 'https://www.hyundai.com/kr/ko/e/customer/center/faq'
driver.get(url)
time.sleep(5)

# 차량 구매 관련 FAQ 구매
result = []

# 기다릴 최대 시간 설정
wait = WebDriverWait(driver, 6)

for menu in range(2, 9):
    print(f"============ [ Menu {menu-1} start ] ============")
    try:

        for page in range(2, 5):

            print(f"----------- [Page {page-1} start] -----------")

            try:
                result.extend(crawl_one_page(driver=driver, wait=wait, action=action))
                
                move_to_next_page(driver=driver, wait=wait, action=action, page=page)
                
            except Exception as e:
                print(f'[ faild page > {page-1} ]')
                print('*'*20)
        

        # 메뉴 버튼을 찾기
        button = driver.find_element(By.XPATH, f'//*[@id="app"]/div[3]/section/div[2]/div/div[2]/section/div/div[1]/div[1]/ul/li[{menu}]/button')
        # 메뉴 버튼이 보이도록 위치 스크롤
        action.move_to_element(button).perform()
        # 메뉴 버튼이 클릭 가능할 때 까지 대기
        button = wait.until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="app"]/div[3]/section/div[2]/div/div[2]/section/div/div[1]/div[1]/ul/li[{menu}]/button')))
        
        # 버튼 클릭
        button.click()
        # 2초 대기
        time.sleep(2)
    
    except Exception as e:
        print(f"[ failed menu > {menu-1} ]")
        print('*'*20)

print(f"[ Crawled Amount : {len(result)} record ]")

============ [ Menu 1 start ] ============
----------- [Page 1 start] -----------
----------- [Page 2 start] -----------
----------- [Page 3 start] -----------
============ [ Menu 2 start ] ============
----------- [Page 1 start] -----------
----------- [Page 2 start] -----------
----------- [Page 3 start] -----------
[ faild FAQ id > 10 ]
********************
============ [ Menu 3 start ] ============
----------- [Page 1 start] -----------
----------- [Page 2 start] -----------
----------- [Page 3 start] -----------
============ [ Menu 4 start ] ============
----------- [Page 1 start] -----------
----------- [Page 2 start] -----------
[ faild page > 2 ]
********************
----------- [Page 3 start] -----------
[ faild page > 3 ]
********************
============ [ Menu 5 start ] ============
----------- [Page 1 start] -----------
----------- [Page 2 start] -----------
----------- [Page 3 start] -----------
============ [ Menu 6 start ] ============
----------- [Page 1 start] -------

In [10]:
df_result = pd.DataFrame(result)

In [11]:
df_result

,quetion,answer
0,"[차량구매]\n보유중인 자동차를 팔고 싶은데, 어떻게 하면 좋을까요?","매입 서비스에 대한 자세한 내용이 궁금하시거나,\n판매 상담을 받으시려면 [Home..."
1,[차량구매]\n포인트는 어떻게 활용하나요?,"포인트는 블루멤버스 사이트 내에서 현금 처럼 사용 하실 수 있으며, 고객님께서 블루..."
2,[차량구매]\n폐차 의뢰 및 신청은 어떻게 하나요?,매도&구매지원 항목의 친환경 폐차 신청으로 들어가셔서 신청하시는 것이 가장 신속하고...
3,[차량구매]\n내차를 좀 더 좋은 값에 판매 할 수 있는 방법은 없나요?,"차량 내부와 트렁크를 청결히 하시고, 차량 외관도 깨끗하게 세차 합니다. 이 때, ..."
4,"[차량구매]\n보유중인 자동차를 팔고 싶은데, 어떻게 하면 좋을까요?",유카의 매입 서비스는 중고차의 합리적인 가치 산정에 대한 환금과 최단 소요 시간으로...
...,...,...
200,[현대 디지털 키]\n휴대폰 nfc기능이 인식이 안돼요. 어떻게 해야하나요?,금속 혹은 두꺼운 핸드폰 케이스를 사용하거나 무선 충전 패드 위에 금속류의 부착물을...
201,[현대 디지털 키]\n핀번호 잊은 경우에는 어떻게 하나요?,"홈페이지에서 마이페이지 상에 PIN 초기화 시, 탈퇴 없이도 본인인증을 하면 PIN..."
202,"[현대 디지털 키]\n안드로이드 OS 버전, 블루투스 버전 확인 방법은 어떻게 되나요?","안드로이드 OS 보는 법, 블루투스 버전 확인은 스마트폰 제조사 홈페이지를 참고해 ..."
203,[현대 디지털 키]\nNFC 카드키 등록은 어떻게 하나요?,NFC카드는 외부에서 구매가 가능해 발렛파킹 등에서 타인의 카드키를 쉽게 등록하는 ...


In [12]:
df_result.to_csv('./data/hyundai_faq.csv', encoding='utf-8')

---

## 제네시스 FAQ 크롤링

In [16]:
chromedriver_autoinstaller.install()
driver = webdriver.Chrome()
action = ActionChains(driver)

# 크롤링할 웹 페이지 URL
url = 'https://www.genesis.com/kr/ko/support/faq.html'
driver.get(url)
time.sleep(3)

result = []

for idx in range(1, 200):
    try :
        result_elem = {}

        # 질문 버튼 찾기
        button = driver.find_element(By.XPATH, f'//*[@id="faq_tab"]/div/div[2]/div[2]/div[1]/div/div/div/div[{idx}]/a')
        # 버튼이 클릭이 가능하도록 해당 위치로 스크롤
        action.move_to_element(button).perform()
        time.sleep(1)
        # 버튼 클릭
        button.click()
        # 2초 대기
        time.sleep(1)


        # 질문 추출
        result_elem['quetion'] = button.text

        # 답변 요소 찾기
        elem = driver.find_element(By.XPATH, f'//*[@id="faq_tab"]/div/div[2]/div[2]/div[1]/div/div/div/div[{idx}]/div/div')
        # 답변 추출
        result_elem['answer'] = elem.text

        result.append(result_elem)
    
    except Exception as e:
        print(f"faild FAQ id > {idx}")
        print("*"*20)

print(f"Crawled Amount > {len(result)} record")

faild FAQ id > 50
********************
faild FAQ id > 194
********************
faild FAQ id > 195
********************
faild FAQ id > 196
********************
faild FAQ id > 197
********************
faild FAQ id > 198
********************
faild FAQ id > 199
********************
Crawled Amount > 192 record


In [18]:
df_result = pd.DataFrame(result)

In [19]:
df_result

,quetion,answer
0,[차량 구매] 제네시스 구입 후 세금계산서 발급받으려고 하는데 어떻게 해야 하나요?...,[개인사업자 및 법인사업자 구입고객]\n2010년 7월 1일부터 세금계산서가 서면이...
1,[차량 구매] 제네시스 차량 구입 시 의무보험료란 금액을 납부하였습니다. 이것은 어...,의무보험은 차량이 출고센터를 동해 출고되어 임시운행 기간 동안에만 유효한 보험입니다...
2,[차량 구매] 일정기간 이상 지난 차량은 자동차세가 경감된다고 하는데 그 내용을 알...,자가용 승용차의 차령 3년이상 차량에 대한 자동차세 경감은 현재에도 시행되고 있는 ...
3,[차량 구매] 부가세 환급을 받을 수 있는 차종 및 조건은 무엇인가요?\n클릭하여 닫기,"개인 또는 법인 사업자 명의로 등록하는 경우 9인승 이상의 차량이나 경차, 화물차 ..."
4,[차량 구매] 차량에 부과되는 세금은 어떤 종류가 있나요?\n클릭하여 닫기,현행 자동차관련 세금항목은 다음과 같습니다.\n\n① 구입단계 : 개별소비세(8인승...
...,...,...
187,[무선 소프트웨어 업데이트(OTA)] 무선 소프트웨어 업데이트로 차량 시스템 업데이...,무선 소프트웨어 업데이트는 대상 고객에 한하여 순차적으로 소프트웨어가 배포됩니다. ...
188,[무선 소프트웨어 업데이트(OTA)] 무선 통신으로 차량에 소프트웨어가 자동으로 다...,무선 소프트웨어 업데이트 진행 사항은 차량 내 인포테인먼트 화면을 통해 확인하실 수...
189,"[무선 소프트웨어 업데이트(OTA)] 다운로드와 설치 준비가 완료되었는데, 업데이트...","12V 배터리 상태 (충전량, 온도 등) 에 따라 업데이트 안내 팝업이 나타나지 않..."
190,[무선 소프트웨어 업데이트(OTA)] 업데이트가 완료되면 업데이트 내용은 어디서 확...,"차량 내 인포테인먼트 화면에서 QR 코드를 확인 가능하며, QR 코드 확인 시 제네..."


In [20]:
df_result.to_csv('./data/genesis_faq.csv', encoding='utf-8')

---

## 쉐보레 FAQ 크롤링

In [29]:
chromedriver_autoinstaller.install()
driver = webdriver.Chrome()
action = ActionChains(driver)

# 크롤링할 웹 페이지 URL
url = 'https://www.chevrolet.co.kr/faq/purchasing-related'
driver.get(url)
time.sleep(3)

result = []

for menu in range(3, 8):
    print(f"========= [menu {menu-1} start] =========")
    try :
        for idx in range(1, 21):
            try:
                result_elem = {}

                

                # 질문 버튼 찾기
                button = driver.find_element(By.XPATH, f'//*[@id="q-main-content"]/adv-grid[5]/adv-col/div/div[{idx}]/div/div[1]')
                # 버튼이 클릭이 가능하도록 해당 위치로 스크롤
                action.move_to_element(button).perform()
                time.sleep(1)
                # 버튼 클릭
                button.click()
                # 1초 대기
                time.sleep(1)


                # 질문 추출
                result_elem['quetion'] = button.text

                # 답변 요소 찾기
                elem = driver.find_element(By.XPATH, f'//*[@id="q-main-content"]/adv-grid[5]/adv-col/div/div[{idx}]/div/div[2]/gb-content-well/adv-grid/adv-col/div/div')
                # 답변 추출
                result_elem['answer'] = elem.text

                result.append(result_elem)
            except Exception as e:
                print(f"failed FAQ id > {idx}")

        # 질문 버튼 찾기
        button = driver.find_element(By.XPATH, f'//*[@id="q-main-content"]/adv-grid[2]/adv-col[1]/div/adv-grid/adv-col[{menu}]/div/a')
        # 버튼이 클릭이 가능하도록 해당 위치로 스크롤
        action.move_to_element(button).perform()
        time.sleep(1)
        # 버튼 클릭
        button.click()
        # 1초 대기
        time.sleep(1)
    except Exception as e:
        print(f"failed menu id > {menu}")

print(f"Crawled Amount > {len(result)}")

========= [menu 2 start] =========
========= [menu 3 start] =========
========= [menu 4 start] =========
failed FAQ id > 13
failed FAQ id > 14
failed FAQ id > 15
failed FAQ id > 16
failed FAQ id > 17
failed FAQ id > 18
failed FAQ id > 19
failed FAQ id > 20
========= [menu 5 start] =========
failed FAQ id > 14
failed FAQ id > 15
failed FAQ id > 16
failed FAQ id > 17
failed FAQ id > 18
failed FAQ id > 19
failed FAQ id > 20
========= [menu 6 start] =========
failed FAQ id > 10
failed FAQ id > 11
failed FAQ id > 12
failed FAQ id > 13
failed FAQ id > 14
failed FAQ id > 15
failed FAQ id > 16
failed FAQ id > 17
failed FAQ id > 18
failed FAQ id > 19
failed FAQ id > 20
Crawled Amount > 74


In [30]:
df_result = pd.DataFrame(result)

df_result

,quetion,answer
0,[구매관련] 근저당 설정 해지 방법 및 절차를 알고싶습니다.,자동차 근저당 설정 해지는 할부를 진행한 할부사의 지점을 방문하여 근저당 설정 해지...
1,[구매관련] 저공해 증명서 재발급 철차 및 장소를 알고싶습니다.,저공해 자동차 증명서는 차량 구입을 담당한 전시장을 통해 재 발급이 가능합니다. \...
2,[구매관련] 재고차 구입 조건시 할인율을 알고싶습니다.,쉐보레 전시장을 통해 자세한 상담 받아주시기 바랍니다.
3,[구매관련] 신차 구입시 출고 예정일 확인 방법을 알고싶습니다.,"출고예정일은 고객님께서 선택하신 차종, 옵션, 색상 등에 따라 다소 차이가 있습니다..."
4,[구매관련] 할부금 중도 상환 방법을 알고싶습니다.,"할부금 중도 상환은 가능하며, 중도상환에 따른 수수료가 발생됩니다.\n 할부금 잔..."
...,...,...
69,[장애인 차량] LPG 구조변경 방법을 알고싶습니다.,가솔린 차량 구매후 LPG 구조변경을 원하는 경우 장애가 있는 당사자 및 가족(직계...
70,[장애인 차량] 장애인과 공동명의 구입시 가능 조건을 알고싶습니다.,"주민등록표상 세대를 함께하는 배우자, 직계존비속, 형제 자매의 경우, 직계비속의 배..."
71,[장애인 차량] 장애인 사망시 차량 승계 방법을 알고싶습니다.,"LPG 전용일 경우에는 가족이 승계할 수는 없으며, 장애인이나 국가유공자에게 판매하..."
72,[장애인 차량] 장애인 구입 혜택은 어떻게 되나요?,장애인 차량 구입시 등급에 따라 세금 혜택이 있습니다.\n\n- 개별소비세 : 면세...


In [32]:
df_result.to_csv('./data/chevrolet_faq.csv', encoding='utf-8')

---

//*[@id="q-main-content"]/adv-grid[2]/adv-col[1]/div/adv-grid/adv-col[3]/div/a

//*[@id="q-main-content"]/adv-grid[2]/adv-col[1]/div/adv-grid/adv-col[4]/div/a

---